嘗試抓取不同頁的留言
> https://www.youtube.com/watch?v=MQH4Rau_F_A

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}


In [3]:
def get_hotel(url):
    #每次调用等待两秒, 避免被ban ip
    time.sleep(2)
    
    uri_list = []
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'html.parser')

    titles = soup.select('div.listing_title > a[target="_blank"]')

    data_list = []

    # 抓該頁各飯店資訊
    for i, title in enumerate(titles):
        # 放肯定會有的資料
        data = {
            'title':title.get_text(),
            'uri': 'https://www.tripadvisor.com.tw' + title.get('href')
        }
        
        uri_list.append(data['uri'])
        data_list.append(data)
    
    return uri_list, data_list

In [4]:
# 抓每頁30間飯店
all_uri = []
url_list = ['https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html'.format(str(i)) for i in range(0,1200,30)]
for k in range(0,10):
    hotel_url, hotels_data = get_hotel(url_list[k])
    all_uri = all_uri + hotel_url

## 如果失敗的網址之後再測試一次or寫入txt

In [27]:
# 抓某一則評論的 user曾訂過哪些飯店
def get_user(url_list):
    users_id = []
    users_profile = []
    comment_date = []
    hotel_name = []
    UID_list = []
    # 抓每間飯店的第一頁留言
    for i in range(len(url_list)):
        time.sleep(2)
        resp = requests.get(url_list[i])
        soup = BeautifulSoup(resp.text,'html.parser')
        try: 
            title = soup.select('#HEADING')[0].get_text()
            # 留言區域
            comment_card = soup.select('.hotels-community-tab-common-Card__card--ihfZB')
            # 整個留言區
            comment_area = soup.select('div[data-test-target="reviews-tab"]')
            # 不同排版的留言區
            comment_area_new = soup.select('#REVIEWS')

            if len(comment_area) >= 1:
                # todo: 換頁用seleniumn
                # 抓第一頁的數則留言
                for j, comment in enumerate(comment_card):
                    # 使用者名稱
                    users_id.append(comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.a.get_text())
                    # 使用者連結
                    users_profile.append('https://www.tripadvisor.com.tw' + comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.a.get('href'))
                    # 評價日期
                    comment_date.append(comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.get_text().split('發表了一則評論，')[1])
                    # 飯店名稱
                    hotel_name.append(title)
                    # todo: member 補 UID 
                    UID_list.append(' ')
                    
            elif len(comment_area_new) >= 1:
                for j, comment in enumerate(comment_area_new[0].select('.review-container')):
                    hotel_name.append(title)
                    # meber UID
                    UID_list.append(comment.select('.member_info')[0].div.get('id'))
                    # 使用者名稱
                    users_id.append(comment.select('.member_info')[0].get_text())
                    # todo: 使用者連結, 用seleniumn click找href
                    users_profile.append(' ')
                    # 評價日期
                    comment_date.append(comment.select('.prw_rup.prw_reviews_stay_date_hsx')[0].get_text())
                    
            else:
                print(url_list[i])
        
        except:
            print('小錯誤： ' + url_list[i])
            continue
            
    return users_id, users_profile, comment_date, hotel_name, UID_list

In [28]:
get_user(all_uri[:10])

([' 88enjoytravel331',
  ' 368florali香港3',
  ' squarekinggink台灣台中5612',
  ' Agnestang06051',
  ' Meimeiyam1',
  'chiusharon',
  'jeffreyca307',
  'SamLuk',
  'terri',
  'Chih',
  'Bonnie Ke',
  'Emma',
  'Cherrie C',
  'Milkc525',
  '7veloce',
  ' Gooolo1',
  ' Minco781',
  ' kayee03161',
  ' Loveen P5',
  ' 256311',
  'Jackie',
  'JZSenpai',
  'Gigi C',
  'Chia-Ping L',
  'Sandy Tan',
  ' Peter C1212',
  ' rickliu6024台灣台北3',
  ' eeLB2015Hong Kong, China927',
  ' MrxandMrsx香港819',
  ' konglinghuashi上海地區上海16',
  'jimcU6787EY',
  '我是旅行全世界房間服務觀察員評比測試員，馬來西亞華人',
  'Dozing Off',
  '小惠 葉',
  'LHChangTC',
  '胡素惠',
  'Krystalching',
  'chengyellowb',
  'Manlamweng',
  'Elaine',
  'Michelle',
  'Chun T',
  'Pan',
  'Kate',
  'Agnesh'],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  'https://www.tripadvisor.com.tw/Profile/chiusharon',
  'https://www.tripadvisor.com.tw/Profile/jeffreyca307',
  'https://www.tripadvisor.com.tw/Profile/SamLuk',
  'https://www.tripadvisor.com.tw/Profile/territerri930',
  'https

300間飯店

In [29]:
%%time
name, profile, date, hotel, UID = get_user(all_uri)

Wall time: 18min 33s


# save as csv

In [30]:
import pandas as pd
data_df = pd.DataFrame()
data_df['username'] = name
data_df['profile_uri'] = profile
data_df['comment_date'] = date
data_df['hotel_name'] = hotel
data_df['UID'] = UID
data_df.head()

,username,profile_uri,comment_date,hotel_name,UID
0,chiusharon,https://www.tripadvisor.com.tw/Profile/chiusharon,8月9日,台北凱達大飯店 (萬華)Caesar Metro Taipei,
1,jeffreyca307,https://www.tripadvisor.com.tw/Profile/jeffrey...,2019年7月,台北凱達大飯店 (萬華)Caesar Metro Taipei,
2,SamLuk,https://www.tripadvisor.com.tw/Profile/SamLuk,2019年7月,台北凱達大飯店 (萬華)Caesar Metro Taipei,
3,terri,https://www.tripadvisor.com.tw/Profile/territe...,2019年7月,台北凱達大飯店 (萬華)Caesar Metro Taipei,
4,Chih,https://www.tripadvisor.com.tw/Profile/Chih201...,2019年7月,台北凱達大飯店 (萬華)Caesar Metro Taipei,


In [31]:
data_df.to_csv('./tapei_tripadvisor_user_from_300hotels.csv', index=False, encoding='utf_8_sig')